# Comparison of Methods to Calculate rH

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from flux_functions import preprocessAmeriflux
from resistance_functions import *

In [2]:
methods = ['thom_1972', 'brutsaert_1982', 'sheppard_1958_v1', 'sheppard_1958_v2', 'owen_thomson_1963', 'zeng_dickinson_1998', 'zilitinkevich_1995', 'kanda_2007', 'kondo_1975', 'log1', 'log10', 'log100'] 

## Charleston Mesquite Woodland

In [3]:
CMW_arg_dict = {'WS_1_col': 'WS_1_1_1', 
            'WS_2_col': 'WS_1_2_1',
            'plant_WS_col': 'WS_1_2_1',
            'z1': 14,
            'z2': 8, 
            'plant_z': 8, #match plant_WS_col
            'USTAR_col': 'USTAR', 
            'H_col': 'H', 
            'plant_TA_col': 'TA_1_2_1', 
            'TC_TA_col': 'IR_tc_ta',
            'PA_col': 'PA',
            'year_col': 'year', 
            'month_col': 'month'}

In [4]:
CMW_Eddy_Covariance_daytime = preprocessAmeriflux('CMW/AMF_US-CMW_BASE_HH_1-5.csv', 
                                                  heights = 3, 
                                                  ustar_thresh = 0.2, 
                                                  time_start = "8:00", 
                                                  time_end = "15:30", 
                                                  time_zone = 'America/Phoenix')

CMW_Eddy_Covariance_daytime['IR_tc_ta'] = (CMW_Eddy_Covariance_daytime.T_CANOPY_1_1_1 + 273.15) - (CMW_Eddy_Covariance_daytime.TA_1_2_1 + 273.15)
CMW_Eddy_Covariance_daytime = CMW_Eddy_Covariance_daytime[CMW_Eddy_Covariance_daytime.year < 2007]
CMW_Eddy_Covariance_daytime = CMW_Eddy_Covariance_daytime.dropna(subset = ['T_CANOPY_1_1_1', 'Rn_G', 'Evap_Frac', 'TA_1_2_1'])

In [5]:
CMW_true_rha = calc_rha_from_temp(pa = CMW_Eddy_Covariance_daytime.PA, 
                                  ta = CMW_Eddy_Covariance_daytime.TA_1_2_1, 
                                  tc_ta = CMW_Eddy_Covariance_daytime.IR_tc_ta, 
                                  h = CMW_Eddy_Covariance_daytime.H, 
                                  rh = CMW_Eddy_Covariance_daytime.RH_1_2_1)

riparian_dict = {}

for m in methods:
    preds = predict_rha(CMW_Eddy_Covariance_daytime, CMW_arg_dict, method = m)
    if ~np.all(CMW_true_rha.index == preds.index):
        print("indexes don't match")
    compare = pd.concat([CMW_true_rha, preds], axis = 1, ignore_index = False).rename(columns = {0: 'true', 1: 'predicted'}).dropna()
    riparian_dict[m] = np.nanmedian(np.abs(compare.true - compare.predicted)).round(2)
    print(m, 'MAE =', np.nanmedian(np.abs(compare.true - compare.predicted)).round(2), 'sm-1, sizes:', len(compare.true), len(compare.predicted))

thom_1972 MAE = 7.25 sm-1, sizes: 7370 7370
brutsaert_1982 MAE = 184.87 sm-1, sizes: 7370 7370
sheppard_1958_v1 MAE = 46.18 sm-1, sizes: 7370 7370
sheppard_1958_v2 MAE = 50.62 sm-1, sizes: 7370 7370
owen_thomson_1963 MAE = 302.57 sm-1, sizes: 7370 7370
zeng_dickinson_1998 MAE = 92.96 sm-1, sizes: 7370 7370
zilitinkevich_1995 MAE = 49.57 sm-1, sizes: 7370 7370
kanda_2007 MAE = 90.2 sm-1, sizes: 7370 7370
kondo_1975 MAE = 51.13 sm-1, sizes: 7370 7370
log1 MAE = 4.42 sm-1, sizes: 7370 7370
log10 MAE = 8.6 sm-1, sizes: 7370 7370
log100 MAE = 19.71 sm-1, sizes: 7370 7370


## Santa Rita Upland Savanna

In [6]:
SRM_arg_dict = {'WS_1_col': 'WS_1_1_1', 
            'WS_2_col': 'WS_1_2_1',
            'plant_WS_col': 'WS_1_2_1', #check sensor
            'z1': 7.8,
            'z2': 3.5, 
            'plant_z': 3.5, #match plant_WS_col
            'USTAR_col': 'USTAR', 
            'H_col': 'H', 
            'plant_TA_col': 'TA_1_2_1', 
            'TC_TA_col': 'IR_tc_ta',
            'PA_col': 'PA',
            'year_col': 'year', 
            'month_col': 'month'}

In [7]:
SRM_Eddy_Covariance_daytime = preprocessAmeriflux('SRM/AMF_US-SRM_BASE_HH_22-5.csv', 
                                                  heights = 2, 
                                                  ustar_thresh = 0.2, 
                                                  time_start = "8:00", 
                                                  time_end = "15:30", 
                                                  time_zone = 'America/Phoenix')

SRM_Eddy_Covariance_daytime['IR_tc_ta'] = (SRM_Eddy_Covariance_daytime.T_CANOPY_1_1_1 + 273.15) - (SRM_Eddy_Covariance_daytime.TA_1_2_1 + 273.15)
SRM_Eddy_Covariance_daytime = SRM_Eddy_Covariance_daytime[~SRM_Eddy_Covariance_daytime.year.isin([2014, 2015, 2020, 2021])]
SRM_Eddy_Covariance_daytime = SRM_Eddy_Covariance_daytime.dropna(subset = ['T_CANOPY_1_1_1', 'Rn_G', 'Evap_Frac', 'TA_1_2_1'])

In [8]:
SRM_true_rha = calc_rha_from_temp(pa = SRM_Eddy_Covariance_daytime.PA, ta = SRM_Eddy_Covariance_daytime.TA_1_2_1, tc_ta = SRM_Eddy_Covariance_daytime.IR_tc_ta, h = SRM_Eddy_Covariance_daytime.H, rh = SRM_Eddy_Covariance_daytime.RH_1_2_1)

upland_dict = {}

for m in methods:
    preds = predict_rha(SRM_Eddy_Covariance_daytime, SRM_arg_dict, method = m)
    if ~np.all(SRM_true_rha.index == preds.index):
        print("indexes don't match")
    compare = pd.concat([SRM_true_rha, preds], axis = 1, ignore_index = False).rename(columns = {0: 'true', 1: 'predicted'}).dropna()
    upland_dict[m] = np.nanmedian(np.abs(compare.true - compare.predicted)).round(2)
    print(m, 'MAE =', np.nanmedian(np.abs(compare.true - compare.predicted)).round(2), 'sm-1, sizes:', len(compare.true), len(compare.predicted))

thom_1972 MAE = 9.1 sm-1, sizes: 14224 14224
brutsaert_1982 MAE = 131.22 sm-1, sizes: 14224 14224
sheppard_1958_v1 MAE = 37.19 sm-1, sizes: 14224 14224
sheppard_1958_v2 MAE = 42.4 sm-1, sizes: 14224 14224
owen_thomson_1963 MAE = 157.22 sm-1, sizes: 14224 14224
zeng_dickinson_1998 MAE = 42.65 sm-1, sizes: 14224 14224
zilitinkevich_1995 MAE = 16.97 sm-1, sizes: 14224 14224
kanda_2007 MAE = 57.4 sm-1, sizes: 14224 14224
kondo_1975 MAE = 43.08 sm-1, sizes: 14224 14224
log1 MAE = 12.43 sm-1, sizes: 14224 14224
log10 MAE = 10.07 sm-1, sizes: 14224 14224
log100 MAE = 16.73 sm-1, sizes: 14224 14224


In [9]:
comparison = pd.DataFrame({'riparian':pd.Series(riparian_dict),'upland':pd.Series(upland_dict)})
comparison['average'] = comparison.mean(axis = 1).round(2)
comparison.sort_values(['average'])

,riparian,upland,average
thom_1972,7.25,9.10,8.18
log1,4.42,12.43,8.43
log10,8.60,10.07,9.34
log100,19.71,16.73,18.22
zilitinkevich_1995,49.57,16.97,33.27
sheppard_1958_v1,46.18,37.19,41.68
sheppard_1958_v2,50.62,42.40,46.51
kondo_1975,51.13,43.08,47.10
zeng_dickinson_1998,92.96,42.65,67.80
kanda_2007,90.20,57.40,73.80


### Check d Values

In [37]:
df_rha = CMW_Eddy_Covariance_daytime
arg_dict = CMW_arg_dict
d_range = slice(0, 10, 0.1)
monthly_d_pred = (df_rha
    .groupby([arg_dict['year_col'], arg_dict['month_col']])
    .apply(func = lambda x: nanbrute(gradient_d, ranges = (d_range,), args = (x[arg_dict['WS_1_col']], x[arg_dict['WS_2_col']], arg_dict['z1'], arg_dict['z2'], x[arg_dict['USTAR_col']],)))
    .to_frame(name = 'd_pred'))